In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

# Harris & hi-Tomasi Corner

In [7]:
noises = ['None','gauss', 's&p', 'poisson', 'speckle']

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[tuple(coords)] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[tuple(coords)] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy
    
    return image



@interact_manual(noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2),
          harris_threshold = (0.01, 0.99, 0.01), harris_block = (2, 10, 1),
          harris_ksize = (0, 10), harris_k=(1, 200, 1))
def features_detect(noise, rotate, scale,
                   harris_threshold, harris_block, harris_ksize, harris_k):
    
    
    harris_k /= 100
    src = cv.imread('/Users/kolsha/Desktop/6LYf-3fIZSQ.jpg')
    #src_gray = cv.medianBlur(src_gray, 5)
    src = cv.cvtColor(src, cv.COLOR_BGR2RGB)
    
    src_gray = cv.cvtColor(src, cv.COLOR_RGB2GRAY)
    
    img = cv.imread('/Users/kolsha/Desktop/HgEFkI4sKwI.jpg')#src.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    num_rows, num_cols = img.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    
    
    img = noisy(noise, img)
    img = img.astype('uint8')
    
    
    img_harris = cv.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    
    img_tomasi = img_harris.copy()
    
    img_gray = cv.cvtColor(img_harris, cv.COLOR_RGB2GRAY)
    
    
    dst_harris = cv.cornerHarris(img_gray, blockSize=harris_block, ksize=harris_ksize, k=harris_k)
    dst_harris = cv.dilate(dst_harris,None)
    
    dst_src_harris = cv.cornerHarris(src_gray, blockSize=harris_block, ksize=harris_ksize, k=harris_k)
    dst_src_harris = cv.dilate(dst_src_harris,None)

    # Threshold for an optimal value, it may vary depending on the image.
    
    img_harris[dst_harris > harris_threshold * dst_harris.max()]=[0,0,255]
    src_harris = src.copy()
    src_harris[dst_src_harris > harris_threshold * dst_src_harris.max()]=[0,0,255]
    
    
    
    maxCorners = 23
    # Parameters for Shi-Tomasi algorithm
    qualityLevel = 0.01
    minDistance = 10
    blockSize = 3
    gradientSize = 3
    useHarrisDetector = False
    k = 0.04
    # Copy the source image
    src_tomasi = src.copy()
    # Apply corner detection
    corners_img = cv.goodFeaturesToTrack(img_gray, maxCorners, qualityLevel, minDistance, None, \
        blockSize=blockSize, gradientSize=gradientSize, useHarrisDetector=useHarrisDetector, k=k)
    
    corners_src = cv.goodFeaturesToTrack(src_gray, maxCorners, qualityLevel, minDistance, None, \
        blockSize=blockSize, gradientSize=gradientSize, useHarrisDetector=useHarrisDetector, k=k)
    # Draw corners detected
    
    radius = 4
    for i in range(corners_img.shape[0]):
        cv.circle(img_tomasi, (corners_img[i,0,0], corners_img[i,0,1]), radius, (0, 0, 255), cv.FILLED)

    for i in range(corners_src.shape[0]):
        cv.circle(src_tomasi, (corners_src[i,0,0], corners_src[i,0,1]), radius, (0, 0, 255), cv.FILLED)

    
    
    titles = ['Src', 'Harris', 'Src', 'Tomasi']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [src_harris, img_harris,
              src_tomasi, img_tomasi]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 20)
    for i in range(len(images)):
        #plt.subplot(2, 3, i+1)
        plt.figure(figsize = (20,20))
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='noise', options=('None', 'gauss', 's&p', 'poisson', 'speckle'), va…